In [23]:
import pandas as pd
import numpy as np
import string
from collections import Counter
from utils import *
from vehicle import Vehicle

import networkx as nx
from bokeh.io import show, output_file, output_notebook
from bokeh.plotting import figure
from bokeh.models.graphs import from_networkx


## Estrazione dati con un piccolo parser dei dati

In [173]:
path='Instances/A1.txt'

In [174]:
#support
count = 0
#first 3 lines of the txt file
number_of_customers=0
one=0
number_of_vehicles=0
#vettori di informazione
vect_cord=[] #lista/vettore di coppiadi coordinate
vect_domanda=[] #linehaul
vect_carico=[] #backhaul
vec_b=[] #vettore delle istanze backhaul
vec_l=[] #vettore delle istanze linehaul
with open(path) as file:
    for line in file:
        #print(line)
        count+=1
        if count  == 1:
            number_of_customers=int((line.split())[0])
        if count  == 2:
            one=int((line.split())[0])
        if count == 3:
            number_of_vehicles=int((line.split())[0])
        if count > 3:
            vect_cord.append((int((line.split())[0]),int((line.split())[1])))
            vect_domanda.append((int((line.split())[2])))
            vect_carico.append((int((line.split())[3]))) 
            if int((line.split())[3]) == 0:
                vec_l.append(count-4) #il numero delle istanze linehaul
            if int((line.split())[3]) > 0:
                vec_b.append(count-4) # il numero delle istanze backhaul

#capacità massima di ogni mezzo
capacity=vect_carico[0]                     

In [175]:
df_data = pd.DataFrame({'': [number_of_customers, number_of_vehicles, capacity]},
                       index=['Number of customers', 'Number of Vehicles', 'Capacity'])

In [177]:
df_instances = pd.DataFrame({'Cordinate X': [ x[0] for x in vect_cord],
                             'Cordinate Y': [ x[1] for x in vect_cord], 
                             'Linehaul': vect_domanda, 
                             'Backhaul' : vect_carico,
                            })

In [178]:
df_data

,
Number of customers,25
Number of Vehicles,8
Capacity,1550


all'indice 0 è riportato il magazzino. dove il campo Backhaul rappresenta la capacità massima

In [179]:
df_instances

,Cordinate X,Cordinate Y,Linehaul,Backhaul
0,12000,16000,0,1550
1,9795,21510,0,549
2,7758,10895,0,558
3,12927,2464,0,851
4,23286,5538,0,388
5,7416,15826,0,194
6,21524,24879,483,0
7,22337,17915,389,0
8,7642,9722,435,0
9,3707,20036,452,0


## Calcolo dei Dati raccolti

In [180]:
distance = distance_matrix(vect_cord)

In [181]:
saving=saving_matrix(distance)

In [182]:
dict_Saving_Order=savingOrder(saving)

## Mappa dei saving ordinati

In [183]:
dict_b, dict_l , dict_merge= route_Linehaul_Backhaul(dict_Saving_Order, vec_b, vec_l)

## Inizializzo la lista delle rotte

In [184]:
list_route = []
set_iniziale = {0}
set_first = {0}
set_visited = {0}
lista_keys= list(dict_l.keys())
for k in lista_keys:
    if len(list_route) < number_of_vehicles and k[0] not in set_iniziale and k[1] not in set_iniziale:
        list_route.append(Vehicle([0, k[0], k[1]], vect_domanda[k[0]]+vect_domanda[k[1]],
                         distance[0][k[0]]+distance[k[0]][k[1]]))
        set_iniziale.add(k[0])
        set_iniziale.add(k[1])
        set_first.add(k[0])
        set_visited.add(k[0])
        set_visited.add(k[1])

In [185]:
remove_first_instance(dict_l, set_first)

In [186]:
np.math.factorial(number_of_customers)/(np.math.factorial(number_of_customers-2)*2)

300.0

In [187]:
c = 0
temp = dict_l.copy()
set_first = {0}
i = 0 
while(i < np.math.factorial(number_of_customers)/(np.math.factorial(number_of_customers-2)*2)):
    lista_keys = list(dict_l.keys())
    flag = True
    for k in lista_keys:
        if 17 in k and list_route[c].route[-1] in k: #controllo cosa c'è quando arriva a 17
            print(f'La route {c}: \t\t {list_route[c].route} \t\t capacità raggiunta: {list_route[c].weight} \t\t pair {k}')
            print('----')
        if list_route[c].route[-1] in k \
        and different(list_route[c].route[-1], k) not in set_visited \
        and list_route[c].weight + vect_domanda[different(list_route[c].route[-1],k)] <= capacity:
                
                list_route[c].add_weight(vect_domanda[different(list_route[c].route[-1],k)])
                list_route[c].add_route(k)           
                set_visited.add(list_route[c].route[-1])
                list_route[c].add_cost(distance[k[0]][k[1]])
                set_first.add(k[0])

                break

    #remove_first_instance(dict_l, set_first)
    c = (c + 1) % number_of_vehicles
    
    i += 1

La route 3: 		 [0, 20, 22] 		 capacità raggiunta: 1196 		 pair (17, 22)
----
La route 4: 		 [0, 14, 21] 		 capacità raggiunta: 1161 		 pair (17, 21)
----
La route 5: 		 [0, 9, 24] 		 capacità raggiunta: 814 		 pair (17, 24)
----
La route 6: 		 [0, 6, 7] 		 capacità raggiunta: 872 		 pair (7, 17)
----
La route 7: 		 [0, 8, 15] 		 capacità raggiunta: 1437 		 pair (15, 17)
----
La route 0: 		 [0, 10, 12, 16] 		 capacità raggiunta: 1169 		 pair (16, 17)
----
La route 1: 		 [0, 19, 23, 18] 		 capacità raggiunta: 1425 		 pair (17, 18)
----
La route 2: 		 [0, 13, 25, 11] 		 capacità raggiunta: 1301 		 pair (11, 17)
----
La route 3: 		 [0, 20, 22] 		 capacità raggiunta: 1196 		 pair (17, 22)
----
La route 4: 		 [0, 14, 21] 		 capacità raggiunta: 1161 		 pair (17, 21)
----
La route 5: 		 [0, 9, 24, 17] 		 capacità raggiunta: 1488 		 pair (17, 24)
----
La route 5: 		 [0, 9, 24, 17] 		 capacità raggiunta: 1488 		 pair (17, 25)
----
La route 5: 		 [0, 9, 24, 17] 		 capacità raggiunta: 1488 		 pair

In [188]:
len(dict_l)

66

In [189]:
for e in list_route:
    e.delivery = list_route[c].weight
    e.weight = 0
    print(e.route)

[0, 10, 12, 16]
[0, 19, 23, 18]
[0, 13, 25, 11]
[0, 20, 22]
[0, 14, 21]
[0, 9, 24, 17]
[0, 6, 7]
[0, 8, 15]


In [192]:
c = 0

while(c < len(list_route)):
    if len(set_visited) <= number_of_customers:
        merge = list(dict_merge.keys())
        back = list(dict_b.keys())
        for k in merge:
            if list_route[c].route[-1] in k and different(list_route[c].route[-1], k) not in set_visited:
                    list_route[c].add_weight(vect_carico[different(list_route[c].route[-1],k)])
                    list_route[c].add_route(k)
                    set_visited.add(list_route[c].route[-1])
                    list_route[c].add_cost(distance[k[0]][k[1]]) 
                    break
    c += 1

In [194]:

c = 0
temp = dict_b.copy()
set_first = {0}
i = 0 
while(i < np.math.factorial(number_of_customers)/(np.math.factorial(number_of_customers-2)*2)):
    lista_keys = list(dict_b.keys())
    flag = True
    for k in lista_keys:
        if list_route[c].route[-1] in k \
        and different(list_route[c].route[-1], k) not in set_visited \
        and list_route[c].weight + vect_carico[different(list_route[c].route[-1],k)] <= capacity:
                
                list_route[c].add_weight(vect_carico[different(list_route[c].route[-1],k)])
                list_route[c].add_route(k)           
                set_visited.add(list_route[c].route[-1])
                list_route[c].add_cost(distance[k[0]][k[1]])
                set_first.add(k[0])
                break

    #remove_first_instance(dict_l, set_first)
    c = (c + 1) % number_of_vehicles
    
    i += 1

In [195]:
for e in list_route:
    print(e.route)

[0, 10, 12, 16, 4]
[0, 19, 23, 18, 3]
[0, 13, 25, 11, 2]
[0, 20, 22, 1]
[0, 14, 21, 5]
[0, 9, 24, 17]
[0, 6, 7]
[0, 8, 15]


In [199]:
len(set_visited) #contando anche il magazzino 

26